# arrival_delay Github Project
Step-by-step this code will go over `scikit-learn` classes and algorithms used to create a model that can predict flight arrival delays.

## Importing Libraries

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

## Retrieving Data
The data used was downloaded from [Kaggle](https://www.kaggle.com/datasets/usdot/flight-delays?select=flights.csv). This project only uses the "airlines" and "flights" data.

In [5]:
airlines = pd.read_csv('airlines.csv')
flights = pd.read_csv('flights.csv')

C:\Users\Taylor\AppData\Local\Temp\ipykernel_21536\1770024609.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights = pd.read_csv('flights.csv')


## Initial Data Analysis

In [7]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [8]:
airlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   IATA_CODE  14 non-null     object
 1   AIRLINE    14 non-null     object
dtypes: object(2)
memory usage: 356.0+ bytes


In [9]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
airlines.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [11]:
flights.shape

(5819079, 31)

In [12]:
airlines.shape

(14, 2)

In [13]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [14]:
airlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   IATA_CODE  14 non-null     object
 1   AIRLINE    14 non-null     object
dtypes: object(2)
memory usage: 356.0+ bytes


In [15]:
flights.describe()

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,...,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
count,5819079.0,5.819079e+06,5.819079e+06,5.819079e+06,5.819079e+06,5.819079e+06,5.732926e+06,5.732926e+06,5.730032e+06,5.730032e+06,...,5.819079e+06,5.726566e+06,5.714008e+06,5.819079e+06,5.819079e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06
mean,2015.0,6.524085e+00,1.570459e+01,3.926941e+00,2.173093e+03,1.329602e+03,1.335204e+03,9.370158e+00,1.607166e+01,1.357171e+03,...,1.493808e+03,1.476491e+03,4.407057e+00,2.609863e-03,1.544643e-02,1.348057e+01,7.615387e-02,1.896955e+01,2.347284e+01,2.915290e+00
std,0.0,3.405137e+00,8.783425e+00,1.988845e+00,1.757064e+03,4.837518e+02,4.964233e+02,3.708094e+01,8.895574e+00,4.980094e+02,...,5.071647e+02,5.263197e+02,3.927130e+01,5.102012e-02,1.233201e-01,2.800368e+01,2.143460e+00,4.816164e+01,4.319702e+01,2.043334e+01
min,2015.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-8.200000e+01,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,-8.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2015.0,4.000000e+00,8.000000e+00,2.000000e+00,7.300000e+02,9.170000e+02,9.210000e+02,-5.000000e+00,1.100000e+01,9.350000e+02,...,1.110000e+03,1.059000e+03,-1.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2015.0,7.000000e+00,1.600000e+01,4.000000e+00,1.690000e+03,1.325000e+03,1.330000e+03,-2.000000e+00,1.400000e+01,1.343000e+03,...,1.520000e+03,1.512000e+03,-5.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,3.000000e+00,0.000000e+00
75%,2015.0,9.000000e+00,2.300000e+01,6.000000e+00,3.230000e+03,1.730000e+03,1.740000e+03,7.000000e+00,1.900000e+01,1.754000e+03,...,1.918000e+03,1.917000e+03,8.000000e+00,0.000000e+00,0.000000e+00,1.800000e+01,0.000000e+00,1.900000e+01,2.900000e+01,0.000000e+00
max,2015.0,1.200000e+01,3.100000e+01,7.000000e+00,9.855000e+03,2.359000e+03,2.400000e+03,1.988000e+03,2.250000e+02,2.400000e+03,...,2.400000e+03,2.400000e+03,1.971000e+03,1.000000e+00,1.000000e+00,1.134000e+03,5.730000e+02,1.971000e+03,1.331000e+03,1.211000e+03


From the analysis, it is evident that "AIRLINE" in the _flights_ dataset corresponds to the "IATA_CODE" in the _airlines_ dataset. Most people do not know what airline is associated with what IATA code off the top of their head. For clarity, these datasets were merged so a single dataframe can be utilized.

In [17]:
df = pd.merge(flights, airlines, left_on='AIRLINE', right_on='IATA_CODE', how='left')

In [18]:
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE_x,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE,AIRLINE_y
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.


Since _flights_ and _airlines_ both contain a column named "AIRLINE", suffixes were appended to the end of that column name. "AIRLINE_x" is a repeat of the IATA codes, and since we already have this column within the dataframe. This column will be dropped. </br>
"AIRLINE_y" consists of the airline names, so this was renamed to just "AIRLINE".

In [20]:
df = df.drop(columns='AIRLINE_x')
df = df.rename(columns={'AIRLINE_y':'AIRLINE'})

## Identifying Target Attribute and Features

Our target attribute (what we are trying to predict) is the arrival delay. It is essential that your target attribute does **not** contain any null values. Most models can not handle null values in the target because it makes it impossible for the model to learn what the correct label should be.</br>
</br>
There are null values within our target attribute, so these will be dropped.

In [23]:
df_cleaned = df.copy()
df_cleaned = df_cleaned.dropna(subset=['ARRIVAL_DELAY'])

There are many columns related to if or why there is was delay. In real-world scenarios, when booking a flight you won't know if there will be a security or weather delay. Because of this, I chose not to use these columns to train this model.</br>
</br>
For simplicity, the features selected were distance, day of week, airline, origin and destination airport. I then separated these features into if they are numerical or categorical. This is essential later on to ensure the correct pipeline is used with the correct features. The categorical columns were then all changed to be strings so the model could process them correctly.

In [25]:
num_cols = ['DISTANCE','DAY_OF_WEEK'] #had to put here bc errored when predicting model bc during training it did not see all the values
cat_cols = ['AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT']

for col in cat_cols:
    df_cleaned[col] = df_cleaned[col].astype(str)

The target attribute, y, and the features, X, were then isolated

In [27]:
X = df_cleaned.drop(['ARRIVAL_DELAY'], axis=1)
y = df_cleaned['ARRIVAL_DELAY']

## Splitting the Data

When splitting the data between test and training sets, a test size of 20% is standard. For reproducibility, a random_state of 42 was selected.

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42) # for reproducibility if following along

## Pipelines

The features were broken up between numerical and categorical types. Some classes only work with certain data types, so a numeric pipeline and a categorical pipeline were made separate. 

In [33]:
num_pipeline = make_pipeline(SimpleImputer(strategy='median'),
                             StandardScaler()) 
cat_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'),
                             OneHotEncoder(drop='first'))

## Column Transformer

A column transformer ensures the data is transformed in the proper order and the steps are executed in the same order everytime.

In [36]:
ct = ColumnTransformer([('num', num_pipeline, num_cols),
                        ('cat', cat_pipeline, cat_cols)])

X_train_transformed = ct.fit_transform(X_train)

## Build Model

Since we are trying to predict arrival delay times, this is considered a regression task. `LinearRegression` was selected as this model, but you could compare how well this model performs against a different model type as well.</br>
</br>
Since this is a relatively big dataset, I chose a lower cv value so it would execute faster.

In [39]:
# initialize
lin_reg = LinearRegression()

#cross validation
cv_scores_lin_reg = cross_val_score(lin_reg,
                                    X_train_transformed,
                                    y_train,
                                    cv=3)

# rmse
rmse_lin_reg = (-cross_val_score(lin_reg,
                                 X_train_transformed,
                                 y_train,
                                 scoring='neg_root_mean_squared_error',
                                 cv=3)).mean()

print("Scores: ", cv_scores_lin_reg)
print('RMSE: ',rmse_lin_reg)

Scores:  [0.01235849 0.01220675 0.01205523]
RMSE:  39.05360100553938


## Fitting Model and Final Evaluations

Now that the model has been created, we will now fit the model and test its performance against testing data. It is crucial to remember to **transform** your X_test. Do not fit_transform. That is for training data.</br>
</br>
I typically initially evaluate my model's performance by observing the Root Mean Squared Error and R2 for regression models.

In [42]:
#fit model 
lin_reg = lin_reg.fit(X_train_transformed, y_train)

# transform test set
X_test_transformed = ct.transform(X_test)

# predict
y_pred = lin_reg.predict(X_test_transformed)

# evaluate
rmse = mean_squared_error(y_test,
                          y_pred,
                          squared=False)
r2 = r2_score(y_test, y_pred)

# print results
print('RMSE: ', rmse)
print('r2: ', r2)

RMSE:  38.936266049224166
r2:  0.012384522854274826


These values were not the best. Since RMSE is sensitive to outliers, I decided to evaluate the Mean Absolute error. This is significantly better than the RMSE value prior.

In [44]:
mae = mean_absolute_error(y_test, y_pred)
print('MAE: ', mae)

MAE:  21.082092198207217


## Conclusion
If I wanted to create a model that was highly accurate and where I know I would have all this provided information, I would select better features. The "delay" provided features would be better features to train a model on. This can be proven using a correlation matrix

In [46]:
def all_numeric_columns(dataframe):
    return dataframe.select_dtypes(include='number').columns.tolist()
    
all_nums = all_numeric_columns(df_cleaned)

corr_matrix = df[all_nums].corr()

corr_matrix["ARRIVAL_DELAY"].sort_values(ascending=False)

ARRIVAL_DELAY          1.000000
DEPARTURE_DELAY        0.944672
AIRLINE_DELAY          0.609351
LATE_AIRCRAFT_DELAY    0.522013
WEATHER_DELAY          0.264799
AIR_SYSTEM_DELAY       0.247187
TAXI_OUT               0.227319
DEPARTURE_TIME         0.159787
WHEELS_OFF             0.155773
TAXI_IN                0.116628
SCHEDULED_DEPARTURE    0.100220
SCHEDULED_ARRIVAL      0.088824
WHEELS_ON              0.058880
ARRIVAL_TIME           0.049876
ELAPSED_TIME           0.029120
FLIGHT_NUMBER          0.018419
SECURITY_DELAY         0.009655
DAY                   -0.003097
AIR_TIME              -0.007108
DAY_OF_WEEK           -0.017027
DISTANCE              -0.025444
SCHEDULED_TIME        -0.030029
MONTH                 -0.036793
YEAR                        NaN
DIVERTED                    NaN
CANCELLED                   NaN
Name: ARRIVAL_DELAY, dtype: float64

Quickly test this theory. I will use the top 3 features from the correlation matrix (besides the target attribute).

In [48]:
# creating new dataframe
df_final = df.copy()
df_final = df_final.dropna(subset=['ARRIVAL_DELAY'])

# selecting new features
num_final = ['DEPARTURE_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY']

# choosing X and y
X_final = df_final.drop(['ARRIVAL_DELAY'], axis=1)
y_final = df_final['ARRIVAL_DELAY']

# splitting data
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X_final,
                                                                            y_final,
                                                                            test_size=0.2,
                                                                            random_state=42)

# pipeline
num_pipeline_final = make_pipeline(SimpleImputer(strategy='median'),
                                   StandardScaler()) 

# column transformer and fit
ct_final = ColumnTransformer([('num_final', num_pipeline_final, num_final)])
X_train_transformed_final = ct_final.fit_transform(X_train_final)

# instantiate, train, and evaluate
lin_reg_final = LinearRegression()
lin_reg_final = lin_reg_final.fit(X_train_transformed_final, y_train_final)
X_test_transformed_final = ct_final.transform(X_test_final)
y_pred_final = lin_reg_final.predict(X_test_transformed_final)

# testing
rmse_final = mean_squared_error(y_test_final,
                                y_pred_final,
                                squared=False)
r2_final = r2_score(y_test_final, y_pred_final)
mae_final = mean_absolute_error(y_test_final, y_pred_final)

# print results
print('RMSE: ', rmse_final)
print('R2: ', r2_final)
print('MAE: ', mae_final)

RMSE:  12.89029003012779
R2:  0.8917557928288771
MAE:  9.058889519244852
